# Halfar Dome Model

Replicating Luke Morris and Jadon Clugston's notebooks
* https://www.cise.ufl.edu/~luke.morris/cism.html
* https://github.com/JuliaComputing/ASKEMDemos/blob/main/Glacial%20Flow/GlacialFlowNotebook.jl

In [93]:
using Catlab
using Catlab.Graphics
using CombinatorialSpaces
using Decapodes

using MLStyle
using MultiScaleArrays
using LinearAlgebra
using OrdinaryDiffEq
using SparseArrays
using Statistics
# using BenchmarkTools

import Pkg
Pkg.add("JLD2")
Pkg.add("GLMakie")
Pkg.add("CairoMakie")
Pkg.add("GeometryBasics")
using GeometryBasics: Point2, Point3

Point2D = Point2{Float64}
Point3D = Point3{Float64}

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`


Point3{Float64} (alias for GeometryBasics.Point{3, Float64})

## Define Models

In [94]:
# Halfar Model

halfar_model = @decapode begin
  h::Form0
  Γ::Form1
  n::Constant

  ḣ == ∂ₜ(h)
  ḣ == ∘(⋆, d, ⋆)(Γ * d(h) * avg₀₁(mag(♯(d(h)))^(n-1)) * avg₀₁(h^(n+2)))
end

# to_graphviz(halfar_model)

Var,type,name
1,Form0,h
2,Form1,Γ
3,Constant,n
4,infer,ḣ
5,infer,mult_1
6,infer,mult_2
7,infer,•1
8,infer,•2
9,infer,•3
10,infer,•4


In [95]:
# Glen's Law Model
glen_model = @decapode begin
    Γ::Form1
    A::Constant
    ρ::Constant
    g::Constant
    n::Constant
  
    Γ == (2/(n+2)) * A * (ρ * g)^n
end

Var,type,name
1,Form1,Γ
2,Constant,A
3,Constant,ρ
4,Constant,g
5,Constant,n
6,infer,•1
7,Literal,2
8,infer,sum_1
9,infer,•2
10,infer,•3


In [96]:
# Dome Model (defined directly)
dome_model = @decapode begin
    h::Form0
    Γ::Form1
    A::Constant
    ρ::Constant
    g::Constant
    n::Constant
    
    # Halfar equation
    ḣ == ∂ₜ(h)
    ḣ == ∘(⋆, d, ⋆)(Γ * d(h) * avg₀₁(mag(♯(d(h)))^(n-1)) * avg₀₁(h^(n+2)))
    
    # Glen's law
    Γ == (2/(n+2)) * A * (ρ * g)^n
end

Var,type,name
1,Form0,h
2,Form1,Γ
3,Constant,A
4,Constant,ρ
5,Constant,g
6,Constant,n
7,infer,ḣ
8,infer,•1
9,infer,mult_1
10,infer,•2


## Model Composition

A complete model can be define at once or be the composition of two smaller models.

The latter case is demonstrated here: a composition UWD is defined and applied to the Halfar and Glen models. 

In [97]:
# Define the composition UWD
dome_model_uwd = @relation () begin
    dynamics(Γ, n)
    stress(Γ, n)
end

Catlab.Programs.RelationalPrograms.UntypedUnnamedRelationDiagram{Symbol, Symbol} {Box:2, Port:4, OuterPort:0, Junction:2, Name:0, VarName:0}
┌─────┬──────────┐
│ Box │     name │
├─────┼──────────┤
│   1 │ dynamics │
│   2 │   stress │
└─────┴──────────┘
┌──────┬─────┬──────────┐
│ Port │ box │ junction │
├──────┼─────┼──────────┤
│    1 │   1 │        1 │
│    2 │   1 │        2 │
│    3 │   2 │        1 │
│    4 │   2 │        2 │
└──────┴─────┴──────────┘
┌──────────┬──────────┐
│ Junction │ variable │
├──────────┼──────────┤
│        1 │        Γ │
│        2 │        n │
└──────────┴──────────┘

In [98]:
# Apply the composition UWD to the two smaller models
dome_model_composite = apex(oapply(dome_model_uwd, [
        Open(halfar_model, [:Γ, :n]), 
        Open(glen_model, [:Γ, :n])
]))

Var,type,name
1,Form0,dynamics_h
2,Form1,Γ
3,Constant,n
4,infer,dynamics_ḣ
5,infer,dynamics_mult_1
6,infer,dynamics_mult_2
7,infer,dynamics_•1
8,infer,dynamics_•2
9,infer,dynamics_•3
10,infer,dynamics_•4


In [99]:
# dome_model == dome_model_composite

## Specify Dimensionality

We need to specify the number of spatial dimensions for which the discrete exterior calculus operators in the model are interpreted.

In [119]:
# 1D
dome_model_1D = expand_operators(dome_model)
infer_types!(dome_model_1D, op1_inf_rules_1D, op2_inf_rules_1D)
resolve_overloads!(dome_model_1D, op1_res_rules_1D, op2_res_rules_1D)

write_json_acset(dome_model_1D, "dome_model_1D.json")
# dome_model_1D = read_json_acset(SummationDecapode{String, String, String}, "dome_model_1D.json")

2448

In [120]:
# 2D
dome_model_2D = expand_operators(dome_model)
infer_types!(dome_model_2D)
resolve_overloads!(dome_model_2D)

write_json_acset(dome_model_2D, "dome_model_2D.json")
# dome_model_2D = read_json_acset(SummationDecapode{String, String, String}, "dome_model_2D.json")

2448

## Define Mesh

The mesh to be used to discretize the domain and the model needs to be defined:
* using helper functions
* uploading a shapefile
* using custom code

In [102]:
# Define a 1D mesh using a helper function

s_prime_1D = EmbeddedDeltaSet1D{Bool, Point2D}()
add_vertices!(s_prime_1D, 20, point = Point2D.(range(0, 10_000, length = 20), 0))
add_edges!(s_prime_1D, 1:nv(s_prime_1D) - 1, 2:nv(s_prime_1D))
orient!(s_prime_1D)
s_1D = EmbeddedDeltaDualComplex1D{Bool, Float64, Point2D}(s_prime_1D)
subdivide_duals!(s_1D, Circumcenter())

# Save both meshes as shapefile

In [103]:
# Define a 2D rectangular triangulated grid using a helper function

function triangulated_grid(max_x, max_y, dx, dy, point_type)

  s = EmbeddedDeltaSet2D{Bool, point_type}()

  # Place equally-spaced points in a max_x by max_y rectangle.
  coords = point_type == Point3D ? map(x -> point_type(x..., 0), Iterators.product(0:dx:max_x, 0:dy:max_y)) : map(x -> point_type(x...), Iterators.product(0:dx:max_x, 0:dy:max_y))
  # Perturb every other row right by half a dx.
  coords[:, 2:2:end] = map(coords[:, 2:2:end]) do row
    if point_type == Point3D
      row .+ [dx/2, 0, 0]
    else
      row .+ [dx/2, 0]
    end
  end
  # The perturbation moved the right-most points past max_x, so compress along x.
  map!(coords, coords) do coord
    if point_type == Point3D
      diagm([max_x/(max_x+dx/2), 1, 1]) * coord
    else
      diagm([max_x/(max_x+dx/2), 1]) * coord
    end
  end

  add_vertices!(s, length(coords), point = vec(coords))

  nx = length(0:dx:max_x)

  # Matrix that stores indices of points.
  idcs = reshape(eachindex(coords), size(coords))
  # Only grab vertices that will be the bottom-left corner of a subdivided square.
  idcs = idcs[begin:end-1, begin:end-1]
  
  # Subdivide every other row along the opposite diagonal.
  for i in idcs[:, begin+1:2:end]
    glue_sorted_triangle!(s, i, i+nx, i+nx+1)
    glue_sorted_triangle!(s, i, i+1, i+nx+1)
  end
  for i in idcs[:, begin:2:end]
    glue_sorted_triangle!(s, i, i+1, i+nx)
    glue_sorted_triangle!(s, i+1, i+nx, i+nx+1)
  end

  # Orient and return.
  s[:edge_orientation]=true
  orient!(s)
  s
end


s_prime_2D_rect = triangulated_grid(10_000, 10_000, 800, 800, Point3D)
s_2D_rect = EmbeddedDeltaDualComplex2D{Bool, Float64, Point3D}(s_prime_2D_rect)
subdivide_duals!(s_2D_rect, Barycenter())

# Save both meshes as shapefile

In [104]:
# Define a 2D sphere in 3D
s_prime_2D_sph = loadmesh(Icosphere(3, 10_000))
s_2D_sph = EmbeddedDeltaDualComplex2D{Bool, Float64, Point3D}(s_prime_2D_sph)
subdivide_duals!(s_2D_sph, Barycenter())

# Save both meshes as shapefile

In [105]:
# Define a 2D teapot in 3D
download("https://graphics.stanford.edu/courses/cs148-10-summer/as3/code/as3/teapot.obj", "teapot.obj")
s_prime_2D_tea = EmbeddedDeltaSet2D("teapot.obj")
s_2D_tea = EmbeddedDeltaDualComplex2D{Bool,Float64,Point3D}(s_prime_2D_tea)
subdivide_duals!(s_2D_tea, Circumcenter())

# Save both meshes as shapefile

# Configure Model

We need to specify values for:
* parameters
* initial conditions
* boundary conditions (optional)

In [106]:
# Parameters
n = 3
ρ = 910
g = 9.8
A = 1e-16

1.0e-16

In [107]:
# Initial conditions

# 1D
h_init_1D = map(point(s_prime_1D)) do (x, _)
        ((7072 - ((x - 5000)^2)) / 9e3 + 2777) / 2777e-1
end

# 2D rectangular triangular grid
h_init_2D_rect = map(point(s_prime_2D_rect)) do (x, y)
  (7072 - ((x - 5000)^2 + (y - 5000)^2)^(1 / 2)) / 9e3 + 10
end

# 2D icosphere in 3D
h_init_2D_sph = map(point(s_prime_2D_sph)) do (x, y, z)
    (z * z) / (10_000 * 10_000)
end

# 2D teapot in 3D
h_init_2D_tea = map(point(s_prime_2D_tea)) do (x, y, z)
    (z * z) * 1_000
end

3644-element Vector{Float32}:
  0.0
  6.561
  6.561
  0.0
  0.0
  0.0
  6.561
  6.561
  6.561
  6.561
 20.735998
 20.735998
 20.735998
  ⋮
  3.8029423
  3.8029423
  0.0
  0.0
  2.9160001
  2.9160001
  3.3635361
  3.3635361
  0.0
  0.0
  0.0
  0.0

In [108]:
# Boundary conditions
# None in this example

## Helper Functions

* `generate(...)`

In [109]:
# Implement DEC operators (♯, ♭, ∧, d, ⋆)
function generate(sd, my_symbol; hodge=GeometricHodge())
  op = @match my_symbol begin
    :♯ => x -> begin
      # This is an implementation of the "sharp" operator from the exterior
      # calculus, which takes co-vector fields to vector fields.
      # This could be up-streamed to the CombinatorialSpaces.jl library. (i.e.
      # this operation is not bespoke to this simulation.)
      e_vecs = map(edges(sd)) do e
        point(sd, sd[e, :∂v0]) - point(sd, sd[e, :∂v1])
      end
      neighbors = map(vertices(sd)) do v
        union(incident(sd, v, :∂v0), incident(sd, v, :∂v1))
      end
      n_vecs = map(neighbors) do es
        [e_vecs[e] for e in es]
      end
      map(neighbors, n_vecs) do es, nvs
        sum([nv*norm(nv)*x[e] for (e,nv) in zip(es,nvs)]) / sum(norm.(nvs))
      end
    end
    :mag => x -> begin
      norm.(x)
    end
    :avg₀₁ => x -> begin
      I = Vector{Int64}()
      J = Vector{Int64}()
      V = Vector{Float64}()
      for e in 1:ne(s)
          append!(J, [s[e,:∂v0],s[e,:∂v1]])
          append!(I, [e,e])
          append!(V, [0.5, 0.5])
      end
      avg_mat = sparse(I,J,V)
      avg_mat * x
    end
    :^ => (x,y) -> x .^ y
    :* => (x,y) -> x .* y
    :show => x -> begin
      @show x
      x
    end
    x => error("Unmatched operator $my_symbol")
  end
  return (args...) -> op(args...)
end

generate (generic function with 1 method)

## Generate and Run Simulation

In [115]:
# Sim parameters
start_time = 0.0
end_time = 8e3

# Map constants to model parameters
constants_and_parameters = (
    n = n,
    stress_ρ = ρ,
    stress_g = g,
    stress_A = 1e-16
)

(n = 3, stress_ρ = 910, stress_g = 9.8, stress_A = [1.0e-16, 1.0e-16, 1.0e-16, 1.0e-16, 1.0e-16, 1.0e-16, 1.0e-16, 1.0e-16, 1.0e-16, 1.0e-16, 1.0e-16, 1.0e-16, 1.0e-16, 1.0e-16, 1.0e-16, 1.0e-16, 1.0e-16, 1.0e-16, 1.0e-16])

In [116]:
# 1D Case

# Map initial conditions to the state variable
u_init = construct(PhysicsState, [VectorForm(h_init_1D)], Float64[], [:dynamics_h])

# Generate simulation
sim = eval(gensim(dome_model_1D, dimension = 1))

# Implement DEC operators on the given mesh
fm = sim(s_1D , generate)

# Precompile
@info("Precompiling Solver")
prob = ODEProblem(fm, u_init, (start_time, start_time + 1e-8), constants_and_parameters)
soln = solve(prob, Tsit5())
soln.retcode != :Unstable || error("Solver was not stable")

# Run
@info("Solving")
prob = ODEProblem(fm, u_init, (start_time, end_time), constants_and_parameters)
soln = solve(prob, Tsit5())
@show soln.retcode
@info("Done")

[ Info: Precompiling Solver


LoadError: ArgumentError: invalid index: nothing of type Nothing

In [112]:
# 2D Rectangular Triangulated Grid Case

# Map initial conditions to the state variable
u_init = construct(PhysicsState, [VectorForm(h_init_2D_rect)], Float64[], [:dynamics_h])

# Generate simulation
sim = eval(gensim(dome_model_2D, dimension = 2))

# Implement DEC operators on the given mesh
fm = sim(s_2D_rect , generate)

# Precompile
@info("Precompiling Solver")
prob = ODEProblem(fm, u_init, (start_time, start_time + 1e-8), constants_and_parameters)
soln = solve(prob, Tsit5())
soln.retcode != :Unstable || error("Solver was not stable")

# Run
@info("Solving")
prob = ODEProblem(fm, u_init, (start_time, end_time), constants_and_parameters)
soln = solve(prob, Tsit5())
@show soln.retcode
@save "2D_rect.jld2" soln
@info("Done")

In [113]:
# 2D Icosphere in 3D Case

# Map initial conditions to the state variable
u_init = construct(PhysicsState, [VectorForm(h_init_2D_sph)], Float64[], [:dynamics_h])

# Generate simulation
sim = eval(gensim(dome_model_2D, dimension = 2))

# Implement DEC operators on the given mesh
fm = sim(s_2D_sph , generate)

# Precompile
@info("Precompiling Solver")
prob = ODEProblem(fm, u_init, (start_time, start_time + 1e-8), constants_and_parameters)
soln = solve(prob, Tsit5())
soln.retcode != :Unstable || error("Solver was not stable")

# Run
@info("Solving")
prob = ODEProblem(fm, u_init, (start_time, end_time), constants_and_parameters)
soln = solve(prob, Tsit5())
@show soln.retcode
@save "2D_sph.jld2" soln
@info("Done")

[ Info: Precompiling Solver


LoadError: ArgumentError: invalid index: nothing of type Nothing

In [117]:
# 2D Teapot in 3D Case

# Map initial conditions to the state variable
u_init = construct(PhysicsState, [VectorForm(h_init_2D_tea)], Float64[], [:dynamics_h])

# Generate simulation
sim = eval(gensim(dome_model_2D, dimension = 2))

# Implement DEC operators on the given mesh
fm = sim(s_2D_tea , generate)

# Precompile
@info("Precompiling Solver")
prob = ODEProblem(fm, u_init, (start_time, start_time + 1e-8), constants_and_parameters)
soln = solve(prob, Tsit5())
soln.retcode != :Unstable || error("Solver was not stable")

# Run
@info("Solving")
prob = ODEProblem(fm, u_init, (start_time, end_time), constants_and_parameters)
soln = solve(prob, Tsit5())
@show soln.retcode
@save "2D_teapot.jld2" soln
@info("Done")

[ Info: Precompiling Solver


LoadError: ArgumentError: invalid index: nothing of type Nothing

## Save Output

In [ ]:
# 